# GSEA for Phase 1 Warm vs. Control Samples 
(generated with featureCounts metaFeature)

In [30]:
library(topGO)
library(KEGGREST)
library(dplyr)
library(clusterProfiler)

clusterProfiler v4.10.0  For help: https://yulab-smu.top/biomedical-knowledge-mining-book/

If you use clusterProfiler in published research, please cite:
T Wu, E Hu, S Xu, M Chen, P Guo, Z Dai, T Feng, L Zhou, W Tang, L Zhan, X Fu, S Liu, X Bo, and G Yu. clusterProfiler 4.0: A universal enrichment tool for interpreting omics data. The Innovation. 2021, 2(3):100141


Attaching package: ‘clusterProfiler’


The following object is masked from ‘package:AnnotationDbi’:

    select


The following object is masked from ‘package:IRanges’:

    slice


The following object is masked from ‘package:S4Vectors’:

    rename


The following object is masked from ‘package:stats’:

    filter




### Prep for analysis

Reading in dataframe of results from DESeq2 of phase 1 warm vs. control samples (ignoring phase 2)

In [2]:
data <- read.csv('/project/pi_sarah_gignouxwolfsohn_uml_edu/julia/CE_MethylRAD_analysis_2018/analysis/significant_genes/FC_sig_p1wc.csv')
head(data)

,X,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,LOC111116054,0.01357395,1.4012644,4.455895,0.3144743,0.7531608,NA
2,LOC111126949,0.01030321,1.4161721,4.455941,0.3178166,0.7506240,NA
3,LOC111110729,0.00000000,NA,NA,NA,NA,NA
4,LOC111112434,0.20274494,0.0000000,4.456174,0.0000000,1.0000000,NA
5,LOC111120752,0.91710261,0.8880094,1.535302,0.5783941,0.5629981,NA
6,LOC111128944,0.00000000,NA,NA,NA,NA,NA


cleaning up dataframe before analysis

In [5]:
# selecting only the columns I want to use
data2 <- select(data, X, log2FoldChange, padj)

# renaming columns
colnames(data2) <- c('geneID', 'lfc', 'padj')

head(data2)

,geneID,lfc,padj
,<chr>,<dbl>,<dbl>
1,LOC111116054,1.4012644,NA
2,LOC111126949,1.4161721,NA
3,LOC111110729,NA,NA
4,LOC111112434,0.0000000,NA
5,LOC111120752,0.8880094,NA
6,LOC111128944,NA,NA


In [11]:
# creating a numeric vector with geneID associated with p-value adjusted
geneList <- data2$padj
names(geneList) <- data2$geneID

# omitting genes with NA values for padj
geneList <- na.omit(geneList)

head(geneList)
length(geneList) # 6,526 genes with a value for p-adjusted

LOC111124802 LOC111101273 LOC111101250 LOC111101262 LOC111133260 LOC111113503 
   0.7331688    0.7331688    0.8010208    0.9615540    0.4716690    0.5885215

[1] 6526

loading in csv file of a table that matches GO ID terms to gene IDs

csv file was obtained from NCBI and manipulated in excel

In [18]:
# reading in csv file
geneID2GO <- read.csv('/project/pi_sarah_gignouxwolfsohn_uml_edu/julia/CE_MethylRAD_analysis_2018/analysis/GO_enrichment_analysis/geneID2GO.txt', sep='\t')

# renaming columns
colnames(geneID2GO) = c('gene','GO_id')

head(geneID2GO)
dim(geneID2GO) # have 22,654 unique genes that have GO annotations

,gene,GO_id
,<chr>,<chr>
1,LOC111133408,GO:2001070
2,LOC111121603,"GO:2000781,GO:2000781"
3,LOC111132389,GO:2000145
4,LOC111115105,"GO:1990904,GO:1990904"
5,LOC111129853,"GO:1990904,GO:1990904"
6,LOC111101512,GO:1990904


[1] 22654     2

creating annotation file for GO objects

In [19]:
# have to create annotation file
geneID2GO <- readMappings(file = '/project/pi_sarah_gignouxwolfsohn_uml_edu/julia/CE_MethylRAD_analysis_2018/analysis/GO_enrichment_analysis/geneID2GO.txt')

# remove header
geneID2GO <- geneID2GO[-1] 

head(geneID2GO)

$LOC111133408
[1] "GO:2001070"

$LOC111121603
[1] "GO:2000781" "GO:2000781"

$LOC111132389
[1] "GO:2000145"

$LOC111115105
[1] "GO:1990904" "GO:1990904"

$LOC111129853
[1] "GO:1990904" "GO:1990904"

$LOC111101512
[1] "GO:1990904"

In [21]:
# creating list of gene names from geneID2GO object
geneNames <- names(geneID2GO)
head(geneNames)

[1] "LOC111133408" "LOC111121603" "LOC111132389" "LOC111115105" "LOC111129853"
[6] "LOC111101512"

for GO analysis, you need to specify a selection function - this tells the program what makes a gene signficant

In [22]:
topDiffGenes <- function(allScore) {
    return(allScore < 0.05) # returns T/F for p-values<0.05
}

x <- topDiffGenes(geneList)
sum(x) ## the number of selected genes

[1] 53

### GO analysis 
**for molecular function**

In [23]:
# creating GO term for molecular function
GOdata_MF <- new("topGOdata", 
               description = 'all genes in phase 1 warm vs. control',
               ontology = "MF", 
               allGenes = geneList, # all annotated genes
               geneSel = topDiffGenes, # tells program how to select a gene of interest (pvalue<0.05)
               annot = annFUN.gene2GO, 
               gene2GO = geneID2GO)

GOdata_MF


Building most specific GOs .....

	( 806 GO terms found. )


Build GO DAG topology ..........

	( 1207 GO terms and 1578 relations. )


Annotating nodes ...............

	( 3695 genes annotated to the GO terms. )




------------------------- topGOdata object -------------------------

 Description:
   -  all genes in phase 1 warm vs. control 

 Ontology:
   -  MF 

 6526 available genes (all genes from the array):
   - symbol:  LOC111124802 LOC111101273 LOC111101250 LOC111101262 LOC111133260  ...
   - score :  0.7331688019 0.7331688019 0.8010207505 0.9615540199 0.4716689617  ...
   - 53  significant genes. 

 3695 feasible genes (genes that can be used in the analysis):
   - symbol:  LOC111101273 LOC111101250 LOC111101262 LOC111133260 LOC111119377  ...
   - score :  0.7331688019 0.8010207505 0.9615540199 0.4716689617 0.5741114271  ...
   - 29  significant genes. 

 GO graph (nodes with at least  1  genes):
   - a graph with directed edges
   - number of nodes = 1207 
   - number of edges = 1578 

------------------------- topGOdata object -------------------------


In [24]:
# creating results object for molecular function
resultKS_MF <- runTest(GOdata_MF, algorithm = "weight01", statistic = "ks")

# putting results into table
tab_MF <- GenTable(GOdata_MF, raw.p.value = resultKS_MF, topNodes = length(resultKS_MF@score), numChar = 120)

head(tab_MF, 15)
dim(tab_MF) #1,207 GO terms


			 -- Weight01 Algorithm -- 

		 the algorithm is scoring 1207 nontrivial nodes
		 parameters: 
			 test statistic: ks
			 score order: increasing


	 Level 13:	1 nodes to be scored	(0 eliminated genes)


	 Level 12:	2 nodes to be scored	(0 eliminated genes)


	 Level 11:	2 nodes to be scored	(2 eliminated genes)


	 Level 10:	12 nodes to be scored	(4 eliminated genes)


	 Level 9:	52 nodes to be scored	(6 eliminated genes)


	 Level 8:	104 nodes to be scored	(42 eliminated genes)


	 Level 7:	226 nodes to be scored	(866 eliminated genes)


	 Level 6:	335 nodes to be scored	(1060 eliminated genes)


	 Level 5:	231 nodes to be scored	(1794 eliminated genes)


	 Level 4:	167 nodes to be scored	(2556 eliminated genes)


	 Level 3:	55 nodes to be scored	(3282 eliminated genes)


	 Level 2:	19 nodes to be scored	(3488 eliminated genes)


	 Level 1:	1 nodes to be scored	(3677 eliminated genes)



,GO.ID,Term,Annotated,Significant,Expected,raw.p.value
,<chr>,<chr>,<int>,<int>,<dbl>,<chr>
1,GO:0004502,kynurenine 3-monooxygenase activity,1,1,0.01,0.00027
2,GO:0004869,cysteine-type endopeptidase inhibitor activity,2,1,0.02,0.00173
3,GO:0004614,phosphoglucomutase activity,2,0,0.02,0.00182
4,GO:0004823,leucine-tRNA ligase activity,1,1,0.01,0.00352
5,GO:0008184,glycogen phosphorylase activity,1,1,0.01,0.00379
6,GO:0008265,Mo-molybdopterin cofactor sulfurase activity,2,0,0.02,0.00771
7,GO:0004781,sulfate adenylyltransferase (ATP) activity,2,0,0.02,0.00890
8,GO:0004020,adenylylsulfate kinase activity,2,0,0.02,0.00890
9,GO:0140658,ATP-dependent chromatin remodeler activity,27,0,0.21,0.01049


[1] 1207    6

GO analysis for **cellular component**

In [26]:
# creating GO term for cellular component
GOdata_CC <- new("topGOdata", 
               description = 'all genes in phase 1 warm vs. control',
               ontology = "CC", 
               allGenes = geneList, # all annotated genes
               geneSel = topDiffGenes, # tells program how to select a gene of interest (pvalue<0.05)
               annot = annFUN.gene2GO, 
               gene2GO = geneID2GO)

GOdata_CC


Building most specific GOs .....

	( 337 GO terms found. )


Build GO DAG topology ..........

	( 546 GO terms and 941 relations. )


Annotating nodes ...............

	( 2920 genes annotated to the GO terms. )




------------------------- topGOdata object -------------------------

 Description:
   -  all genes in phase 1 warm vs. control 

 Ontology:
   -  CC 

 6526 available genes (all genes from the array):
   - symbol:  LOC111124802 LOC111101273 LOC111101250 LOC111101262 LOC111133260  ...
   - score :  0.7331688019 0.7331688019 0.8010207505 0.9615540199 0.4716689617  ...
   - 53  significant genes. 

 2920 feasible genes (genes that can be used in the analysis):
   - symbol:  LOC111101273 LOC111133260 LOC111119377 LOC111101799 LOC111117672  ...
   - score :  0.7331688019 0.4716689617 0.5741114271 0.5609155014 0.7998816576  ...
   - 19  significant genes. 

 GO graph (nodes with at least  1  genes):
   - a graph with directed edges
   - number of nodes = 546 
   - number of edges = 941 

------------------------- topGOdata object -------------------------


In [27]:
# creating results object for cellular component
resultKS_CC <- runTest(GOdata_CC, algorithm = "weight01", statistic = "ks")

# putting results into table
tab_CC <- GenTable(GOdata_CC, raw.p.value = resultKS_CC, topNodes = length(resultKS_CC@score), numChar = 120)

head(tab_CC, 15)
dim(tab_CC) #546 GO terms


			 -- Weight01 Algorithm -- 

		 the algorithm is scoring 546 nontrivial nodes
		 parameters: 
			 test statistic: ks
			 score order: increasing


	 Level 13:	3 nodes to be scored	(0 eliminated genes)


	 Level 12:	3 nodes to be scored	(0 eliminated genes)


	 Level 11:	15 nodes to be scored	(5 eliminated genes)


	 Level 10:	49 nodes to be scored	(6 eliminated genes)


	 Level 9:	78 nodes to be scored	(33 eliminated genes)


	 Level 8:	85 nodes to be scored	(106 eliminated genes)


	 Level 7:	75 nodes to be scored	(291 eliminated genes)


	 Level 6:	79 nodes to be scored	(597 eliminated genes)


	 Level 5:	70 nodes to be scored	(731 eliminated genes)


	 Level 4:	49 nodes to be scored	(1461 eliminated genes)


	 Level 3:	37 nodes to be scored	(1550 eliminated genes)


	 Level 2:	2 nodes to be scored	(1962 eliminated genes)


	 Level 1:	1 nodes to be scored	(2884 eliminated genes)



,GO.ID,Term,Annotated,Significant,Expected,raw.p.value
,<chr>,<chr>,<int>,<int>,<dbl>,<chr>
1,GO:0070772,PAS complex,2,0,0.01,0.0038
2,GO:0005891,voltage-gated calcium channel complex,6,0,0.04,0.0108
3,GO:0070469,respirasome,7,0,0.05,0.0111
4,GO:0016593,Cdc73/Paf1 complex,3,0,0.02,0.0118
5,GO:0005882,intermediate filament,1,0,0.01,0.0192
6,GO:0016605,PML body,3,0,0.02,0.0193
7,GO:0031966,mitochondrial membrane,46,0,0.30,0.0240
8,GO:0005764,lysosome,19,1,0.12,0.0248
9,GO:0034464,BBSome,6,0,0.04,0.0311


[1] 546   6

GO analysis for **biological process**

In [28]:
# creating GO term for biological process
GOdata_BP <- new("topGOdata", 
               description = 'all genes in phase 1 warm vs. control',
               ontology = "BP", 
               allGenes = geneList, # all annotated genes
               geneSel = topDiffGenes, # tells program how to select a gene of interest (pvalue<0.05)
               annot = annFUN.gene2GO, 
               gene2GO = geneID2GO)

GOdata_BP


Building most specific GOs .....

	( 747 GO terms found. )


Build GO DAG topology ..........

	( 1977 GO terms and 4012 relations. )


Annotating nodes ...............

	( 2537 genes annotated to the GO terms. )




------------------------- topGOdata object -------------------------

 Description:
   -  all genes in phase 1 warm vs. control 

 Ontology:
   -  BP 

 6526 available genes (all genes from the array):
   - symbol:  LOC111124802 LOC111101273 LOC111101250 LOC111101262 LOC111133260  ...
   - score :  0.7331688019 0.7331688019 0.8010207505 0.9615540199 0.4716689617  ...
   - 53  significant genes. 

 2537 feasible genes (genes that can be used in the analysis):
   - symbol:  LOC111133260 LOC111101799 LOC111117672 LOC111114201 LOC111114212  ...
   - score :  0.4716689617 0.5609155014 0.7998816576 0.5093044557 0.9411499616  ...
   - 19  significant genes. 

 GO graph (nodes with at least  1  genes):
   - a graph with directed edges
   - number of nodes = 1977 
   - number of edges = 4012 

------------------------- topGOdata object -------------------------


In [29]:
# creating results object for biological process
resultKS_BP <- runTest(GOdata_BP, algorithm = "weight01", statistic = "ks")

# putting results into table
tab_BP <- GenTable(GOdata_BP, raw.p.value = resultKS_BP, topNodes = length(resultKS_BP@score), numChar = 120)

head(tab_BP, 15)
dim(tab_BP) #1,977 GO terms


			 -- Weight01 Algorithm -- 

		 the algorithm is scoring 1977 nontrivial nodes
		 parameters: 
			 test statistic: ks
			 score order: increasing


	 Level 16:	1 nodes to be scored	(0 eliminated genes)


	 Level 15:	4 nodes to be scored	(0 eliminated genes)


	 Level 14:	20 nodes to be scored	(1 eliminated genes)


	 Level 13:	48 nodes to be scored	(8 eliminated genes)


	 Level 12:	99 nodes to be scored	(48 eliminated genes)


	 Level 11:	147 nodes to be scored	(116 eliminated genes)


	 Level 10:	199 nodes to be scored	(252 eliminated genes)


	 Level 9:	259 nodes to be scored	(405 eliminated genes)


	 Level 8:	276 nodes to be scored	(705 eliminated genes)


	 Level 7:	283 nodes to be scored	(969 eliminated genes)


	 Level 6:	257 nodes to be scored	(1329 eliminated genes)


	 Level 5:	201 nodes to be scored	(1590 eliminated genes)


	 Level 4:	112 nodes to be scored	(2159 eliminated genes)


	 Level 3:	57 nodes to be scored	(2339 eliminated genes)


	 Level 2:	13 nodes to be sco

,GO.ID,Term,Annotated,Significant,Expected,raw.p.value
,<chr>,<chr>,<int>,<int>,<dbl>,<chr>
1,GO:0030705,cytoskeleton-dependent intracellular transport,8,1,0.06,0.00079
2,GO:2000036,regulation of stem cell population maintenance,2,1,0.01,0.00186
3,GO:0003352,regulation of cilium movement,1,1,0.01,0.00276
4,GO:0006429,leucyl-tRNA aminoacylation,1,1,0.01,0.00315
5,GO:0006661,phosphatidylinositol biosynthetic process,14,0,0.10,0.00404
6,GO:0006893,Golgi to plasma membrane transport,3,0,0.02,0.00524
7,GO:0032481,positive regulation of type I interferon production,1,0,0.01,0.00788
8,GO:0002218,activation of innate immune response,3,0,0.02,0.00789
9,GO:0000103,sulfate assimilation,2,0,0.01,0.00797


[1] 1977    6

## clusterProfiler for KEGG analysis
creating KEGG object for downstream analysis

In [31]:
head(data2)

,geneID,lfc,padj
,<chr>,<dbl>,<dbl>
1,LOC111116054,1.4012644,NA
2,LOC111126949,1.4161721,NA
3,LOC111110729,NA,NA
4,LOC111112434,0.0000000,NA
5,LOC111120752,0.8880094,NA
6,LOC111128944,NA,NA


In [36]:
data3 <- na.omit(data2, data2$lfc)
data3$geneID

[1] "LOC111124802" "LOC111101273" "LOC111101250" "LOC111101262"
   [5] "LOC111133260" "LOC111113503" "LOC111119377" "LOC111101799"
   [9] "LOC111117672" "LOC111114053" "LOC111114201" "LOC111114212"
  [13] "LOC111138315" "LOC111138321" "LOC111121058" "LOC111108901"
  [17] "LOC111130095" "LOC111136780" "LOC111133372" "LOC111137104"
  [21] "LOC111115406" "LOC111102448" "LOC111113346" "LOC111100652"
  [25] "LOC111134791" "LOC111128971" "LOC111104153" "LOC111121733"
  [29] "LOC111131721" "LOC111131752" "LOC111131766" "LOC111131682"
  [33] "LOC111112387" "LOC111114799" "LOC111129446" "LOC111100566"
  [37] "LOC111136882" "LOC111136873" "LOC111100662" "LOC111100672"
  [41] "LOC111103288" "LOC111133651" "LOC111105807" "LOC111129837"
  [45] "LOC111126102" "LOC111129935" "LOC111111995" "LOC111134929"
  [49] "LOC111103139" "LOC111119020" "LOC111125333" "LOC111136966"
  [53] "LOC111112068" "LOC111124863" "LOC111124850" "LOC111132898"
  [57] "LOC111101139" "LOC111101156" "LOC111108648" "LOC111117581"
  [61] "LOC111127966" "LOC111135129" "LOC111118316" "LOC111134171"
  [65] "LOC111111295" "LOC111130258" "LOC111124761" "LOC111121500"
  [69] "LOC111130637" "LOC111130647" "LOC111130653" "LOC111130687"
  [73] "LOC111130696" "LOC111121491" "LOC111103323" "LOC111137796"
  [77] "LOC111105035" "LOC111125571" "LOC111125589" "LOC111128222"
  [81] "LOC111128188" "LOC111128319" "LOC111128314" "LOC111100637"
  [85] "LOC111111564" "LOC111137561" "LOC111119531" "LOC111127317"
  [89] "LOC111123891" "LOC111130860" "LOC111127104" "LOC111100578"
  [93] "LOC111130886" "LOC111137420" "LOC111137407" "LOC111111015"
  [97] "LOC111114229" "LOC111102556" "LOC111118371" "LOC111130926"
 [101] "LOC111130935" "LOC111135084" "LOC111123961" "LOC111109472"
 [105] "LOC111134266" "LOC111132215" "LOC111138488" "LOC111126382"
 [109] "LOC111126395" "LOC111126874" "LOC111110435" "LOC111132155"
 [113] "LOC111108767" "LOC111131814" "LOC111127446" "LOC111102092"
 [117] "LOC111099502" "LOC111125391" "LOC111118239" "LOC111135155"
 [121] "LOC111138286" "LOC111125536" "LOC111099029" "LOC111108095"
 [125] "LOC111100900" "LOC111111855" "LOC111120335" "LOC111137504"
 [129] "LOC111112613" "LOC111110056" "LOC111110068" "LOC111123556"
 [133] "LOC111109163" "LOC111137591" "LOC111112589" "LOC111126353"
 [137] "LOC111104824" "LOC111132822" "LOC111121313" "LOC111134625"
 [141] "LOC111131233" "LOC111138167" "LOC111126487" "LOC111124937"
 [145] "LOC111131355" "LOC111135116" "LOC111131259" "LOC111117980"
 [149] "LOC111101737" "LOC111113022" "LOC111128838" "LOC111134535"
 [153] "LOC111131329" "LOC111131341" "LOC111104327" "LOC111104302"
 [157] "LOC111104344" "LOC111131346" "LOC111104366" "LOC111104335"
 [161] "LOC111101542" "LOC111116519" "LOC111116615" "LOC111116630"
 [165] "LOC111128797" "LOC111110283" "LOC111126321" "LOC111126328"
 [169] "LOC111133771" "LOC111117050" "LOC111114913" "LOC111102545"
 [173] "LOC111099722" "LOC111124829" "LOC111108643" "LOC111131148"
 [177] "LOC111131506" "LOC111135310" "LOC111100092" "LOC111105961"
 [181] "LOC111105980" "LOC111122823" "LOC111103080" "LOC111106021"
 [185] "LOC111122911" "LOC111102738" "LOC111102762" "LOC111136085"
 [189] "LOC111131850" "LOC111120886" "LOC111114379" "LOC111104401"
 [193] "LOC111111194" "LOC111125438" "LOC111113174" "LOC111110355"
 [197] "LOC111110282" "LOC111112004" "LOC111112262" "LOC111110995"
 [201] "LOC111106169" "LOC111110814" "LOC111099102" "LOC111126296"
 [205] "LOC111133146" "LOC111126159" "LOC111132855" "LOC111101764"
 [209] "LOC111111600" "LOC111111651" "LOC111111579" "LOC111111571"
 [213] "LOC111128027" "LOC111133491" "LOC111102183" "LOC111119998"
 [217] "LOC111129328" "LOC111126993" "LOC111123242" "LOC111137396"
 [221] "LOC111136967" "LOC111113171" "LOC111136976" "LOC111123818"
 [225] "LOC111123848" "LOC111123884" "LOC111100029" "LOC111126972"
 [229] "LOC111101598" "LOC111116607" "LOC111119989" "LOC111122145"
 [233] "LOC111122190" "LOC111126636" "LOC111126660" "LOC111126577"
 [237] "LOC111126694" "LOC111126605" "LOC111126727

for KEGG analysis, doing ranked-list so need to omit NA values for log2FoldChange and rank in order from highest to lowest - also need entrez gene ID instead of accession number

In [42]:
david_df <- read.csv('/project/pi_sarah_gignouxwolfsohn_uml_edu/julia/CE_MethylRAD_analysis_2018/analysis/KEGG_pathway/FC_p1_warm.v.control/entrez_conversion.txt', sep='\t')
# only selecting columns that I need
david_df <- select(david_df, From, To)
# renaming columns for merge
colnames(david_df) = c('geneID', 'entrez_ID')
head(david_df)

,geneID,entrez_ID
,<chr>,<int>
1,LOC111134684,111134684
2,LOC111108397,111108397
3,LOC111107066,111107066
4,LOC111134686,111134686
5,LOC111107067,111107067
6,LOC111133354,111133354


need to match up dataframes to convert ensembl accession IDs to entrez ID

In [43]:
merge_df <- merge(david_df, data3, by = "geneID", all = TRUE)
head(merge_df)

,geneID,entrez_ID,lfc,padj
,<chr>,<int>,<dbl>,<dbl>
1,LOC111099029,111099029,-1.2510075,0.3089792
2,LOC111099040,111099040,-3.1995344,0.2312968
3,LOC111099050,111099050,0.2171810,0.8585328
4,LOC111099062,111099062,0.6505828,0.6912762
5,LOC111099067,111099067,0.3936674,0.7008011
6,LOC111099073,111099073,-0.7789759,0.4230513


In [45]:
# only selecting the entrez ID and log2FoldChange column
kegg_df <- select(merge_df, entrez_ID, lfc)
head(kegg_df)

,entrez_ID,lfc
,<int>,<dbl>
1,111099029,-1.2510075
2,111099040,-3.1995344
3,111099050,0.2171810
4,111099062,0.6505828
5,111099067,0.3936674
6,111099073,-0.7789759


In [46]:
# Create a vector of the gene unuiverse
kegg_gene_list <- kegg_df$lfc

# Name vector with ENTREZ ids
names(kegg_gene_list) <- kegg_df$entrez_ID

# omit any NA values 
kegg_gene_list<-na.omit(kegg_gene_list)

# sort the list in decreasing order (required for clusterProfiler)
kegg_gene_list = sort(kegg_gene_list, decreasing = TRUE)

head(kegg_gene_list)
class(kegg_gene_list)

111102074 111122982 111108982 111137731 111131233 111106540 
 7.512910  6.016896  4.817644  4.512312  4.486583  4.405996

[1] "numeric"

In [47]:
kegg_organism = "cvn"
kk2 <- gseKEGG(geneList     = kegg_gene_list,
               organism     = kegg_organism,
               nPerm        = 10000,
               minGSSize    = 1,
               maxGSSize    = 800,
               pvalueCutoff = 1, # if this is set to 1, see more pathways, but 0.05 is statistically signif.
               pAdjustMethod = "BH", # Benjamini–Hochberg FDR (false discover rate)
               scoreType = "pos",
               keyType       = "kegg")

Reading KEGG annotation online: "https://rest.kegg.jp/link/cvn/pathway"...

Reading KEGG annotation online: "https://rest.kegg.jp/list/pathway/cvn"...

preparing geneSet collections...

GSEA analysis...

Warning message in .GSEA(geneList = geneList, exponent = exponent, minGSSize = minGSSize, :
“We do not recommend using nPerm parameter incurrent and future releases”
Warning message in fgsea(pathways = geneSets, stats = geneList, nperm = nPerm, minSize = minGSSize, :
“You are trying to run fgseaSimple. It is recommended to use fgseaMultilevel. To run fgseaMultilevel, you need to remove the nperm argument in the fgsea function call.”
Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are duplicate gene names, fgsea may produce unexpected results.”
leading edge analysis...

done...



In [48]:
kk2_df <- as.data.frame(kk2)
kk2_df$Description <- sub(" -.*", "", kk2_df$Description)
head(kk2_df) # actually shows the entire df since there's only 5 pathways with pval<0.05

,ID,Description,setSize,enrichmentScore,NES,pvalue,p.adjust,qvalue,rank,leading_edge,core_enrichment
,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
cvn01040,cvn01040,Biosynthesis of unsaturated fatty acids,9,0.7130094,2.801751,0.00869913,0.5252975,0.5072124,47,"tags=22%, list=1%, signal=22%",111113990/111115744
cvn00790,cvn00790,Folate biosynthesis,5,0.8421314,2.784385,0.00549945,0.5252975,0.5072124,199,"tags=40%, list=3%, signal=39%",111100388/111102230
cvn00592,cvn00592,alpha-Linolenic acid metabolism,7,0.7291269,2.671202,0.01579842,0.5252975,0.5072124,646,"tags=43%, list=10%, signal=39%",111113990/111115744/111127642
cvn01212,cvn01212,Fatty acid metabolism,31,0.4604414,2.344583,0.01519848,0.5252975,0.5072124,56,"tags=13%, list=1%, signal=13%",111127947/111113990/111115744/111103990
cvn00750,cvn00750,Vitamin B6 metabolism,6,0.6983531,2.442111,0.03979602,0.5657980,0.5463186,992,"tags=50%, list=15%, signal=42%",111102977/111112584/111115125
cvn00510,cvn00510,N-Glycan biosynthesis,19,0.4985095,2.335440,0.02899710,0.5657980,0.5463186,1093,"tags=32%, list=17%, signal=26%",111101820/111137033/111136571/111134828/111124588/111124498


getting completely different results from htseq-counts results... 